## Import libs

In [1]:
%pip install datefinder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import datefinder

## Load data from Wikipedia

In [3]:
website = requests.get("https://en.wikipedia.org/wiki/List_of_Olympic_medalists_in_athletics_(women)")

soup = bs(website.content)

html = soup.prettify();

##  Fetch and clean data to store in Pandas DataFrame

In [4]:
final_rows = []
medalists_df = pd.DataFrame()
for index, value in enumerate(soup.select('h3')):
    table = value.find_next_sibling('table')
    tr_list = []
    if not table is None:
        '''
        Due to incoherent structures of tables,
        it is necessary to remove column names
        '''
        try:
            tr_list = table.find_all('tbody')[1].find_all('tr')
        except:
            tr_list = table.find_all('tbody')[0].find_all('tr')

    url = "https://en.wikipedia.org"
    row_list = []
    disc_column = value.get_text().replace('[edit]', '')

    '''
    Remove relay disciplines because this analysis aimed to individual records
    '''
    if 'relay' in disc_column:
        continue

    def has_numbers(input_string):
        input_string = input_string.find('a')
        if input_string is None:
            return False
        return any(char.isdigit() for char in input_string.get_text())

    def handle_blank(birthday):
        if not birthday:
            return 'No data'
        else:
            return birthday

    def has_vacants(entry):
        if 'Vacant' in entry.get_text():
            new_rows.append('none awarded')
            for i in range(2):
                new_rows.append('n/a')
            return True
    
    def has_non_awarded(entry):
        if not (entry.find('i') is None):
            new_rows.append('none awarded')
            return True

    def get_birthday(athlete):
        athlete = athlete.find('a')
        if not (athlete is None):
            if not (athlete.get('title') is None):
                if not (athlete.get_text()[0].isdigit()):
                    actual_url = url + athlete.get('href')
                    new_soup = requests.get(actual_url)
                    athlete_page = bs(new_soup.content)

                    '''
                    Cleaning out unneccessary html tags
                    '''
                    for tag in athlete_page.find_all(["sup", "span"]):
                        tag.decompose()
                    try:
                        '''
                        Date of birth is stored in the infobox on the right side of the page
                        '''
                        table_date = athlete_page.find('table', class_="infobox vcard").find('th', string='Born')
                        if not (table_date is None):
                            bday = table_date.find_next_sibling().get_text()
                            sep = '('
                            bday = bday.split(sep, 1)[0]
                            
                            '''
                            Wikipedia has a lot of incosistency between pages, thats why
                            external library is needed to commonize dates
                            '''
                            extracted = list(datefinder.find_dates(bday))
                            return extracted[0].strftime('%d-%m-%Y')
                    except:
                        return 'No data'


    for tr in tr_list:
        td = tr.find_all('td')

        entries = [e for e in td]

        '''
        To drop ex aequo medalists
        '''
        if len(entries) == 1:
            continue

        new_rows = [disc_column]
        for entry in entries:
            if has_vacants(entry) or has_non_awarded(entry):
                continue
            if not has_numbers(entry):
                '''
                To have seperate column for athlete name and her nationality
                '''
                splitted = entry.get_text().split("\xa0")
                new_rows.append(str(splitted[0]).strip())
                if len(splitted) > 1:
                    new_rows.append(str(splitted[1]).strip())
                else:
                    new_rows.append(entry.get_text)
                '''
                There is no date of birth in the main table so it is neccessary to
                open sub-webpage for each athlete and fetch it
                '''
                new_rows.append(handle_blank(get_birthday(entry)))
            elif str(entry.get_text())[0].isdigit():
                '''
                When column starts with digit it is always year of the games combined
                with city where it took place
                '''
                entry = str(entry.get_text())
                splitted = entry.split(' ', 1)
                new_rows.append(splitted[0])
                '''
                Neccessary cleaning of data
                '''
                new_rows.append(splitted[1].replace('details', '').strip())
            else:
                new_rows.append(entry.get_text())
        row_list.append(new_rows)
    '''
    Creating seperate DataFrame for each discipline and merging in each iteration
    '''
    df = pd.DataFrame(row_list)
    medalists_df = pd.concat([medalists_df, df])
    final_rows.append(row_list)
column_names = ['Discipline', 'Year of Games', 'Place of Games', 'Gold medalist', 'Country (Gold)',
                'Date of birth (Gold)', 'Silver medalist', 'Country (Silver)', 'Date of birth (Silver)',
                'Bronze medalist', 'Country (Bronze)', 'Date of birth (Bronze)']
medalists_df.columns = column_names

## Additional cleaning of DataFrame

In [5]:
medalists_df = medalists_df[medalists_df['Country (Gold)'].str.contains('included')==False]
medalists_df.replace(to_replace=[None], value='n/a', inplace=True)
medalists_df.head(10)

,Discipline,Year of Games,Place of Games,Gold medalist,Country (Gold),Date of birth (Gold),Silver medalist,Country (Silver),Date of birth (Silver),Bronze medalist,Country (Bronze),Date of birth (Bronze)
0,100 metres,1928,Amsterdam,Betty Robinson,United States,23-08-1911,Fanny Rosenfeld,Canada,28-12-1904,Ethel Smith,Canada,05-07-1907
1,100 metres,1932,Los Angeles,Stanisława Walasiewicz,Poland,03-04-1911,Hilda Strike,Canada,01-09-1910,Wilhelmina von Bremen,United States,13-08-1909
2,100 metres,1936,Berlin,Helen Stephens,United States,03-02-1918,Stanisława Walasiewicz,Poland,03-04-1911,Käthe Krauß,Germany,29-11-1906
3,100 metres,1948,London,Fanny Blankers-Koen,Netherlands,26-04-1918,Dorothy Manley,Great Britain,29-04-1927,Shirley Strickland,Australia,18-07-1925
4,100 metres,1952,Helsinki,Marjorie Jackson,Australia,13-09-1931,Daphne Hasenjager,South Africa,02-07-1929,Shirley Strickland de la Hunty,Australia,18-07-1925
5,100 metres,1956,Melbourne,Betty Cuthbert,Australia,20-04-1938,Christa Stubnick,United Team of Germany,12-12-1933,Marlene Matthews,Australia,14-07-1934
6,100 metres,1960,Rome,Wilma Rudolph,United States,23-06-1940,Dorothy Hyman,Great Britain,09-05-1941,Giuseppina Leone,Italy,21-12-1934
7,100 metres,1964,Tokyo,Wyomia Tyus,United States,29-08-1945,Edith McGuire,United States,03-06-1944,Ewa Kłobukowska,Poland,01-10-1946
8,100 metres,1968,Mexico City,Wyomia Tyus,United States,29-08-1945,Barbara Ferrell,United States,28-07-1947,Irena Szewińska,Poland,24-05-1946
9,100 metres,1972,Munich,Renate Stecher,East Germany,12-05-1950,Raelene Boyle,Australia,24-06-1951,Silvia Chivás,Cuba,10-09-1954


## Extend DataFrame with additional columns (age when awarded)

In [6]:
import numpy as np
import datetime
def extract_year(value):
    return int(value.split("-")[2].strip(" "))

medalists_df['Age (Gold)'] = medalists_df['Year of Games'].astype('int') - medalists_df['Date of birth (Gold)'].apply(lambda x: '0' if x == 'n/a' or x == 'No data' else f"{extract_year(x)}").astype('int')
medalists_df['Age (Silver)'] = medalists_df['Year of Games'].astype('int') - medalists_df['Date of birth (Silver)'].apply(lambda x: '0' if x == 'n/a' or x == 'No data' else f"{extract_year(x)}").astype('int')
medalists_df['Age (Bronze)'] = medalists_df['Year of Games'].astype('int') - medalists_df['Date of birth (Bronze)'].apply(lambda x: '0' if x == 'n/a' or x == 'No data' else f"{extract_year(x)}").astype('int')
medalists_df['Age (Gold)'] = medalists_df['Age (Gold)'].apply(lambda x: np.nan if x >1000 else x)
medalists_df['Age (Silver)'] = medalists_df['Age (Silver)'].apply(lambda x: np.nan if x >1000 else x)
medalists_df['Age (Bronze)'] = medalists_df['Age (Bronze)'].apply(lambda x: np.nan if x >1000 else x)
medalists_df = medalists_df[['Discipline', 'Year of Games', 'Place of Games', 'Gold medalist', 'Country (Gold)',
                'Date of birth (Gold)', 'Age (Gold)', 'Silver medalist', 'Country (Silver)', 'Date of birth (Silver)','Age (Silver)',
                'Bronze medalist', 'Country (Bronze)', 'Date of birth (Bronze)', 'Age (Bronze)']]
medalists_df.head(10)

,Discipline,Year of Games,Place of Games,Gold medalist,Country (Gold),Date of birth (Gold),Age (Gold),Silver medalist,Country (Silver),Date of birth (Silver),Age (Silver),Bronze medalist,Country (Bronze),Date of birth (Bronze),Age (Bronze)
0,100 metres,1928,Amsterdam,Betty Robinson,United States,23-08-1911,17.0,Fanny Rosenfeld,Canada,28-12-1904,24.0,Ethel Smith,Canada,05-07-1907,21.0
1,100 metres,1932,Los Angeles,Stanisława Walasiewicz,Poland,03-04-1911,21.0,Hilda Strike,Canada,01-09-1910,22.0,Wilhelmina von Bremen,United States,13-08-1909,23.0
2,100 metres,1936,Berlin,Helen Stephens,United States,03-02-1918,18.0,Stanisława Walasiewicz,Poland,03-04-1911,25.0,Käthe Krauß,Germany,29-11-1906,30.0
3,100 metres,1948,London,Fanny Blankers-Koen,Netherlands,26-04-1918,30.0,Dorothy Manley,Great Britain,29-04-1927,21.0,Shirley Strickland,Australia,18-07-1925,23.0
4,100 metres,1952,Helsinki,Marjorie Jackson,Australia,13-09-1931,21.0,Daphne Hasenjager,South Africa,02-07-1929,23.0,Shirley Strickland de la Hunty,Australia,18-07-1925,27.0
5,100 metres,1956,Melbourne,Betty Cuthbert,Australia,20-04-1938,18.0,Christa Stubnick,United Team of Germany,12-12-1933,23.0,Marlene Matthews,Australia,14-07-1934,22.0
6,100 metres,1960,Rome,Wilma Rudolph,United States,23-06-1940,20.0,Dorothy Hyman,Great Britain,09-05-1941,19.0,Giuseppina Leone,Italy,21-12-1934,26.0
7,100 metres,1964,Tokyo,Wyomia Tyus,United States,29-08-1945,19.0,Edith McGuire,United States,03-06-1944,20.0,Ewa Kłobukowska,Poland,01-10-1946,18.0
8,100 metres,1968,Mexico City,Wyomia Tyus,United States,29-08-1945,23.0,Barbara Ferrell,United States,28-07-1947,21.0,Irena Szewińska,Poland,24-05-1946,22.0
9,100 metres,1972,Munich,Renate Stecher,East Germany,12-05-1950,22.0,Raelene Boyle,Australia,24-06-1951,21.0,Silvia Chivás,Cuba,10-09-1954,18.0


## Export to csv

In [7]:
medalists_df.to_csv('medalists.csv', index=False)

## Export to json

In [14]:
import json
medalists = medalists_df.to_json(orient='split')
parsed = json.loads(medalists)
json.dumps(parsed, indent=4);

## Store data in Postgresql

In [9]:
%pip install sqlalchemy
%pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import psycopg2
from sqlalchemy import create_engine
  
  
conn_string = '****************'
  
db = create_engine(conn_string)
conn = db.connect()

medalists_df.to_sql('data', con=conn, if_exists='replace', index=False, method='multi')

conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
  
sql1 = '''select * from data;'''
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)
  
conn.close()

('100 metres', '1928', 'Amsterdam', 'Betty Robinson', 'United States', '23-08-1911', 17.0, 'Fanny Rosenfeld', 'Canada', '28-12-1904', 24.0, 'Ethel Smith', 'Canada', '05-07-1907', 21.0)
('100 metres', '1932', 'Los Angeles', 'Stanisława Walasiewicz', 'Poland', '03-04-1911', 21.0, 'Hilda Strike', 'Canada', '01-09-1910', 22.0, 'Wilhelmina von Bremen', 'United States', '13-08-1909', 23.0)
('100 metres', '1936', 'Berlin', 'Helen Stephens', 'United States', '03-02-1918', 18.0, 'Stanisława Walasiewicz', 'Poland', '03-04-1911', 25.0, 'Käthe Krauß', 'Germany', '29-11-1906', 30.0)
('100 metres', '1948', 'London', 'Fanny Blankers-Koen', 'Netherlands', '26-04-1918', 30.0, 'Dorothy Manley', 'Great Britain', '29-04-1927', 21.0, 'Shirley Strickland', 'Australia', '18-07-1925', 23.0)
('100 metres', '1952', 'Helsinki', 'Marjorie Jackson', 'Australia', '13-09-1931', 21.0, 'Daphne Hasenjager', 'South Africa', '02-07-1929', 23.0, 'Shirley Strickland de la Hunty', 'Australia', '18-07-1925', 27.0)
('100 metr